# Вебинар 6. Двухуровневые модели рекомендаций


Код для src, utils, metrics вы можете скачать из [этого](https://github.com/geangohn/recsys-tutorial) github репозитория

In [1]:
!pip install implicit

In [2]:
!pip install lightgbm

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

In [3]:
data = pd.read_csv('retail_train.csv')
item_features = pd.read_csv('product.csv')
user_features = pd.read_csv('hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)


# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [4]:
n_items_before = data_train_lvl_1['item_id'].nunique()

#попробуем поварьировать take_n_popular=1000, 2000, 500
data_train_lvl_1 = prefilter_items(data_train_lvl_1, item_features=item_features, take_n_popular=500)

n_items_after = data_train_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 83685 to 501


In [5]:
recommender = MainRecommender(data_train_lvl_1)

### Уровень 1

A) Попробуйте различные варианты генерации кандидатов. Какие из них дают наибольший recall@k ?
- Пока пробуем отобрать 50 кандидатов (k=50)
- Качество измеряем на data_val_lvl_1: следующие 6 недель после трейна

Дают ли own recommendtions + top-popular лучший recall?  

B)* Как зависит recall@k от k? Постройте для одной схемы генерации кандидатов эту зависимость для k = {20, 50, 100, 200, 500}  
C)* Исходя из прошлого вопроса, как вы думаете, какое значение k является наиболее разумным?


In [9]:
result_lvl_1 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_1.columns=['user_id', 'actual']
result_lvl_1.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."


In [10]:
result = result_lvl_1[result_lvl_1['user_id'] == 2375]
result.head(2) 

,user_id,actual
2044,2375,"[835300, 848029, 862714, 896613, 923746, 93246..."


Сделаем тоже самое по всем user:

In [11]:
stats = data_train_lvl_1.head(2000) #для быстрого подсчета метрик

stats['als'] = stats['user_id'].apply(lambda x: recommender.get_als_recommendations(x))
stats['own'] = stats['user_id'].apply(lambda x: recommender.get_own_recommendations(x))
stats['own500'] = stats['user_id'].apply(lambda x: recommender.get_own_recommendations(x,N=500))

stats['similar_items'] = stats['user_id'].apply(lambda x: recommender.get_similar_items_recommendation(x))
#stats['similar_users'] = stats['user_id'].apply(lambda x: recommender.get_similar_users_recommendation(x))

stats = result_lvl_1.merge(stats, on=['user_id'])
print('presicion')
print('als:', stats.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean())
print('own:', stats.apply(lambda row: precision_at_k(row['own'], row['actual']), axis=1).mean())
print('similar_items:', stats.apply(lambda row: precision_at_k(row['similar_items'], row['actual']), axis=1).mean())
#print(stats.apply(lambda row: precision_at_k(row['similar_users'], row['actual']), axis=1).mean())

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

presicion
als: 0.12432432432432401
own: 0.25531914893616797
similar_items: 0.08855664174813173


Для own мы уже достигли минимально необходимого уровня точности.

In [12]:
print('recall')
print('als:', stats.apply(lambda row: recall_at_k(row['als'], row['actual']), axis=1).mean())
print('own:', stats.apply(lambda row: recall_at_k(row['own'], row['actual']), axis=1).mean())
print('own20:', stats.apply(lambda row: recall_at_k(row['own500'], row['actual'],k=20), axis=1).mean())
print('own50:', stats.apply(lambda row: recall_at_k(row['own500'], row['actual'],k=50), axis=1).mean())
print('own100:', stats.apply(lambda row: recall_at_k(row['own500'], row['actual'],k=100), axis=1).mean())
print('own200:', stats.apply(lambda row: recall_at_k(row['own500'], row['actual'],k=200), axis=1).mean())
print('own500:', stats.apply(lambda row: recall_at_k(row['own500'], row['actual'],k=500), axis=1).mean())


print('similar_items:', stats.apply(lambda row: recall_at_k(row['similar_items'], row['actual']), axis=1).mean())


recall
als: 0.009382466453703973
own: 0.017809540289863666
own20: 0.04335885796134176
own50: 0.06656905134305338
own100: 0.0840028767867324
own200: 0.10017956981808716
own500: 0.1276657478020531
similar_items: 0.009075568931981822


### Уровень 2.

Обучите модель 2-ого уровня, при этом:
    - Добавьте минимум по 2 фичи для юзера, товара и пары юзер-товар
    - Измерьте отдельно precision@5 модели 1-ого уровня и двухуровневой модели на data_val_lvl_2
    - Вырос ли precision@5 при использовании двухуровневой модели?

In [13]:
# your_code
users_lvl_2 = pd.DataFrame(data_train_lvl_2['user_id'].unique())
users_lvl_2.columns = ['user_id']

# Пока только warm start
train_users = data_train_lvl_1['user_id'].unique()
users_lvl_2 = users_lvl_2[users_lvl_2['user_id'].isin(train_users)]

users_lvl_2['candidates'] = users_lvl_2['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=50))


In [14]:
users_lvl_2.head(2)

,user_id,candidates
0,2070,"[1016800, 917033, 926905, 913210, 5569374, 933..."
1,2021,"[1013928, 6534077, 896862, 1000753, 883932, 10..."


In [15]:
s = users_lvl_2.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'item_id'

users_lvl_2 = users_lvl_2.drop('candidates', axis=1).join(s)
users_lvl_2['flag'] = 1

users_lvl_2.head(4)

,user_id,item_id,flag
0,2070,1016800,1
0,2070,917033,1
0,2070,926905,1
0,2070,913210,1


In [16]:
users_lvl_2.shape[0]

107550

In [17]:
users_lvl_2['user_id'].nunique()

2151

In [18]:
107550 / 2151

50.0

Добавим в качестве дополнительной фичи текущую цену товара

In [19]:
#Цена
data_train_lvl_2['price'] = data_train_lvl_2['sales_value'] / (np.maximum(data_train_lvl_2['quantity'], 1))
targets_lvl_2 = data_train_lvl_2[['user_id', 'item_id', 'price']].copy()
targets_lvl_2['target'] = 1  # тут только покупки 

targets_lvl_2 = users_lvl_2.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')

targets_lvl_2['target'].fillna(0, inplace= True)
targets_lvl_2.drop('flag', axis=1, inplace=True)
targets_lvl_2.shape[0]

112308

In [20]:
targets_lvl_2.head(10)

,user_id,item_id,price,target
0,2070,1016800,NaN,0.0
1,2070,917033,NaN,0.0
2,2070,926905,NaN,0.0
3,2070,913210,3.59,1.0
4,2070,5569374,NaN,0.0
5,2070,933067,2.50,1.0
6,2070,838186,3.99,1.0
7,2070,12810391,17.59,1.0
8,2070,999270,NaN,0.0
9,2070,857503,NaN,0.0


In [21]:
targets_lvl_2['target'].mean()

0.1283523880756491

In [22]:
item_features = pd.read_csv('product.csv')
# column processing
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


Добавим в качестве дополнительных фичей товара: 1) общую сумму продаж 2) общее кол-во продажи с учетом размера продажи 3) общее кол-во продаж без учета размера продажи

In [23]:
popular = data.groupby('item_id')['sales_value'].sum().reset_index() #общая сумма покупок
#соединить!
item_features = item_features.merge(popular, on='item_id')

sum_quantity = data.groupby('item_id')['quantity'].sum().reset_index() #общее кол-во покупок 
item_features = item_features.merge(sum_quantity, on='item_id')

count_sales = data.groupby('item_id')['quantity'].count().reset_index() #общее кол-во покупок 
item_features = item_features.merge(count_sales, on='item_id')

item_features.head(10)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,sales_value,quantity_x,quantity_y
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB,20.94,6,3
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,,0.99,1,1
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,,1.59,1,1
3,26190,69,GROCERY,Private,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ,1.54,1,1
4,26355,69,GROCERY,Private,COOKIES/CONES,SPECIALTY COOKIES,14 OZ,1.98,2,1
5,26426,69,GROCERY,Private,SPICES & EXTRACTS,SPICES & SEASONINGS,2.5 OZ,2.29,1,1
6,26540,69,GROCERY,Private,COOKIES/CONES,TRAY PACK/CHOC CHIP COOKIES,16 OZ,2.79,3,2
7,26601,69,DRUG GM,Private,VITAMINS,VITAMIN - MINERALS,300CT(1),7.59,1,1
8,26636,69,PASTRY,Private,BREAKFAST SWEETS,SW GDS: SW ROLLS/DAN,,2.50,1,1
9,26691,16,GROCERY,Private,PNT BTR/JELLY/JAMS,HONEY,12 OZ,8.37,3,2


Добавим в качестве дополнительных фичей пользователя:
    1) общую сумму покупок
    2) общее кол-во покупок с учетом размера покупки
    3) общее кол-во покупок без учета размера покупки

In [24]:
user_features = pd.read_csv('hh_demographic.csv')
# column processing
user_features.columns = [col.lower() for col in user_features.columns]
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

popular = data.groupby('user_id')['sales_value'].sum().reset_index() #общая сумма покупок
#соединить!
user_features = user_features.merge(popular, on='user_id')

sum_quantity = data.groupby('user_id')['quantity'].sum().reset_index() #общее кол-во покупок 
user_features = user_features.merge(sum_quantity, on='user_id')

count_sales = data.groupby('user_id')['quantity'].count().reset_index() #общее кол-во покупок 
user_features = user_features.merge(count_sales, on='user_id')

user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id,sales_value,quantity_x,quantity_y
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1,3959.91,1844,1589
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7,2865.89,1304,1072


In [25]:
targets_lvl_2 = targets_lvl_2.merge(item_features, on='item_id', how='left')
targets_lvl_2 = targets_lvl_2.merge(user_features, on='user_id', how='left')

targets_lvl_2.head(2)

,user_id,item_id,price,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,...,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,sales_value_y,quantity_x_y,quantity_y_y
0,2070,1016800,NaN,0.0,103,GROCERY,National,SOFT DRINKS,SOFT DRINKS 12/18&15PK CAN CAR,12 OZ,...,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,6981.86,121765.0,2415.0
1,2070,917033,NaN,0.0,103,GROCERY,National,SOFT DRINKS,SOFT DRINKS 12/18&15PK CAN CAR,12 OZ,...,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,6981.86,121765.0,2415.0


In [26]:
targets_lvl_2.shape[1]

23

In [27]:
X_train = targets_lvl_2.drop('target', axis=1)
y_train = targets_lvl_2[['target']]

In [28]:
cat_feats = ['manufacturer',
 'department',
 'brand',
 'commodity_desc',
 'sub_commodity_desc',
 'curr_size_of_product',
 'age_desc',
 'marital_status_code',
 'income_desc',
 'homeowner_desc',
 'hh_comp_desc',
 'household_size_desc',
 'kid_category_desc']
X_train[cat_feats] = X_train[cat_feats].astype('category')

cat_feats

['manufacturer',
 'department',
 'brand',
 'commodity_desc',
 'sub_commodity_desc',
 'curr_size_of_product',
 'age_desc',
 'marital_status_code',
 'income_desc',
 'homeowner_desc',
 'hh_comp_desc',
 'household_size_desc',
 'kid_category_desc']

In [29]:
lgb = LGBMClassifier(objective='binary', max_depth=7, categorical_column=cat_feats)
lgb.fit(X_train, y_train)

train_preds = lgb.predict(X_train)#[:,1]
#train_preds = lgb.predict_proba(X_train)[:,1]

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:863: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


проверим точность на тренировочных данных

In [30]:
i = 0
j = 0
n = 0
y = np.array(y_train)
for train_pred in train_preds:
    if train_pred == y[n]:
        i += 1
    else:
        j += 1
    n += 1    
print(i)
print(j)

112308
0


In [31]:
precision = i / len(train_preds)
precision

1.0

расчет точности на тестовых данных:

In [32]:
n_items_before = data_val_lvl_2['item_id'].nunique()

data_val_lvl_2 = prefilter_items(data_val_lvl_2, item_features=item_features, take_n_popular=500)

n_items_after = data_val_lvl_2['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 24329 to 501


In [33]:
# your_code
users_lvl_3 = pd.DataFrame(data_val_lvl_2['user_id'].unique())
users_lvl_3.columns = ['user_id']

# Пока только warm start
train_users = data_train_lvl_1['user_id'].unique()
users_lvl_3 = users_lvl_3[users_lvl_3['user_id'].isin(train_users)]

users_lvl_3['candidates'] = users_lvl_3['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=50))


In [34]:
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
s = users_lvl_3.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'item_id'

users_lvl_3 = users_lvl_3.drop('candidates', axis=1).join(s)
users_lvl_3['flag'] = 1

users_lvl_3.head(4)

,user_id,item_id,flag
0,338,1026118,1
0,338,1082185,1
0,338,1004906,1
0,338,1009187,1


In [35]:
data_val_lvl_2['price'] = data_val_lvl_2['sales_value'] / (np.maximum(data_val_lvl_2['quantity'], 1))

targets_lvl_3 = data_val_lvl_2[['user_id', 'item_id', 'price']].copy()
targets_lvl_3['target'] = 1  # тут только покупки 

targets_lvl_3 = users_lvl_3.merge(targets_lvl_3, on=['user_id', 'item_id'], how='left')

targets_lvl_3['target'].fillna(0, inplace= True)
targets_lvl_3.drop('flag', axis=1, inplace=True)
targets_lvl_3.shape[0]


101425

In [36]:
targets_lvl_3 = targets_lvl_3.merge(item_features, on='item_id', how='left')
targets_lvl_3 = targets_lvl_3.merge(user_features, on='user_id', how='left')


In [37]:
X_test = targets_lvl_3.drop('target', axis=1)
y_test = targets_lvl_3[['target']]

In [38]:
print(len(X_test))

101425


In [39]:
cat_feats = ['manufacturer',
 'department',
 'brand',
 'commodity_desc',
 'sub_commodity_desc',
 'curr_size_of_product',
 'age_desc',
 'marital_status_code',
 'income_desc',
 'homeowner_desc',
 'hh_comp_desc',
 'household_size_desc',
 'kid_category_desc']
X_test[cat_feats] = X_test[cat_feats].astype('category')

cat_feats

['manufacturer',
 'department',
 'brand',
 'commodity_desc',
 'sub_commodity_desc',
 'curr_size_of_product',
 'age_desc',
 'marital_status_code',
 'income_desc',
 'homeowner_desc',
 'hh_comp_desc',
 'household_size_desc',
 'kid_category_desc']

In [40]:
test_preds = lgb.predict(X_test)


In [41]:
i = 0
j = 0
n = 0
y = np.array(y_test)
for test_pred in test_preds:
    if test_pred == y[n]:
        i += 1
    else:
        j += 1
    n += 1    
print(i)
print(j)

101425
0


In [42]:
precision = i / len(test_preds)
precision

1.0

In [43]:
test_preds = lgb.predict_proba(X_test)[:,1]
test_preds

array([9.99964775e-01, 9.99964775e-01, 9.99964775e-01, ...,
       5.78654121e-06, 5.78654121e-06, 5.78654121e-06])

In [44]:
X_test['test_preds'] = test_preds
RESULTS = ['user_id', 'item_id', 'test_preds']
X_test[RESULTS]

,user_id,item_id,test_preds
0,338,1026118,0.999965
1,338,1026118,0.999965
2,338,1026118,0.999965
3,338,1082185,0.000006
4,338,1004906,0.999965
5,338,1009187,0.000006
6,338,961979,0.000006
7,338,1068292,0.000006
8,338,1135258,0.000006
9,338,6548453,0.000006


### Финальный проект

Мы уже прошли всю необходимуб теорию для финального проекта. Проект осуществляется на данных из вебинара (данные считаны в начале ДЗ).
Рекомендуем вам **начать делать проект сразу после этого домашнего задания**
- Целевая метрика - precision@5. Порог для уcпешной сдачи проекта precision@5 > 25%
- Будет public тестовый датасет, на котором вы сможете измерять метрику
- Также будет private тестовый датасет для измерения финального качества
- НЕ обязательно, но крайне желательно использовать 2-ух уровневые рекоммендательные системы в проекте
- Вы сдаете код проекта в виде github репозитория и csv файл с рекомендациями 